## Imports and setup

In [ ]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from pathlib import Path

In [ ]:
plt.style.use('seaborn-paper')
plt.style.use('seaborn-colorblind')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Parameters

In [ ]:
efficiencies = [0.4, 0.7, 0.9]
zeniths = [10.00, 23.630, 32.06, 43.20]

erange = (15*u.GeV, 25*u.TeV)

angresylim = (0.0, 0.5)
effareaylim = (1e3, 1e6)
eneylim = (-0.16, 0.7)

In [ ]:
opt = dict(elinewidth=0.8)

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors

----

In [ ]:
import plot_irfs

In [ ]:
import datetime
def print_date(filename):
    print(datetime.datetime.fromtimestamp(filename.stat().st_ctime))

In [ ]:
indir = 'irfs'
outdir = 'irfs_plots'
Path(outdir).mkdir(exist_ok=True)
print(list(Path(indir).iterdir()))
print(list(Path(outdir).iterdir()))

### MAGIC performances

Extracted from _The major upgrade of the MAGIC telescopes, Part II: A performance study using observations of the Crab Nebula_, J. Aleksíc et al, 2015

See https://zenodo.org/record/8027794



In [ ]:
from astropy.table import QTable
magic_table_a2 = QTable.read('magic_perfs/magic_2015_table_a2.ecsv')
magic_table_a3 = QTable.read('magic_perfs/magic_2015_table_a3.ecsv')
magic_table_a4_zen3045 = QTable.read('magic_perfs/magic_2015_table_a4_30zen45.ecsv')
magic_table_a4_lowzen = QTable.read('magic_perfs/magic_2015_table_a4_zen30.ecsv')

magic_table_a2['E_center'] = np.sqrt(magic_table_a2['E_min'] * magic_table_a2['E_max'])
magic_table_a3['E_center'] = np.sqrt(magic_table_a3['E_min'] * magic_table_a3['E_max'])

# Zenith 10°

In [ ]:
zen = 10.00

fig, axes = plt.subplots(3, 1, figsize=(4, 6), sharex=True)


for ii, efficiency in enumerate(efficiencies):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    print_date(filename)
    plot_irfs.plot_angular_resolution_from_file(filename, ax=axes[0],
                                                ls='-',
                                                **opt
                                               )
    plot_irfs.plot_effective_area_from_file(filename, ax=axes[1],
                                            ls='-',
                                            label=f'efficiency = {100*efficiency:.0f}%',
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                               ls='-',
                                               color=colors[ii],
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=colors[ii],
                                         ls='dotted',
                                         **opt
                                        )
    

plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                           ls='-',
                                           color=colors[ii],
                                           label=r'Energy resolution ($\sigma_{68}$)',
                                           **opt
                                              )
plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=colors[ii],
                                         ls='dotted',
                                     label=r'Energy bias (median)',
                                     **opt
                                    )


for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))
    
    
    
## MAGIC
mopt = dict(color='slategray', alpha=1.0)
axes[0].plot(magic_table_a4_lowzen['E'].to(u.TeV), magic_table_a4_lowzen['theta_68'],
             label='MAGIC, 2013, Zenith < 30deg',  **mopt)
axes[2].plot(magic_table_a2['E_center'].to(u.TeV), magic_table_a2['bias'],
             ls='dotted',  **mopt)
axes[2].plot(magic_table_a2['E_center'].to(u.TeV), magic_table_a2['σ'],
             label='MAGIC, 2013, Zenith < 30°)', ls='-', **mopt)



axes[0].set_title(f'Zenith {zen:.2f}°')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[0].set_ylim(*angresylim)
axes[1].set_ylim(*effareaylim)
axes[2].set_ylim(*eneylim)
axes[2].set_ylabel(r'$\Delta E/E_{true}$')
axes[2].legend()

axes[1].grid(True, which='both', axis='x')
axes[1].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')
axes[1].set_axisbelow(True)

axes[1].legend(loc='lower right')
plt.tight_layout()

outfile = Path(outdir, f'irfs_eff_zen_{zen:.2f}.png')
plt.savefig(outfile, dpi=200)

In [ ]:
for efficiency in efficiencies:
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    plot_irfs.plot_theta_cut_per_energy(filename, ls='-', label=f"efficiency={efficiency*100:.0f}%")
plt.grid(True)
outfile = Path(outdir, f'theta_cuts_{zen:.2f}.png')
plt.savefig(outfile, dpi=200)

In [ ]:
for efficiency in efficiencies:
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    plot_irfs.plot_gh_cut_per_energy(filename, ls='-', label=f"efficiency={efficiency*100:.0f}%")
plt.grid(True)
outfile = Path(outdir, f'gh_cuts_{zen:.2f}.png')
plt.savefig(outfile, dpi=200)

## All nodes, efficiency=0.7

In [ ]:
colors

In [ ]:
efficiency = 0.7
zen = 10.0

fig, axes = plt.subplots(3, 1, figsize=(4, 6), sharex=True)
fig.subplots_adjust(hspace=0) 

calt = [ colors[1], colors[3], colors[4], colors[5]]


for ii, zen in enumerate(zeniths):
    color = calt[ii]
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    print_date(filename)
    plot_irfs.plot_angular_resolution_from_file(filename, ax=axes[0],
                                                ls='-',
                                                color=color,
                                                **opt
                                               )
    plot_irfs.plot_effective_area_from_file(filename, ax=axes[1],
                                            ls='-',
                                            label=f'zenith = {zen:.2f}°',
                                            color=color,
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                               ls='-',
                                               color=color,
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=color,
                                         ls='dotted',
                                         **opt
                                        )
    

plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                           ls='-',
                                           color=color,
                                           label=r'Energy resolution ($\sigma_{68}$)',
                                           **opt
                                              )
plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=color,
                                         ls='dotted',
                                     label=r'Energy bias (median)',
                                     **opt
                                    )


## MAGIC
axes[0].plot(magic_table_a4_lowzen['E'].to(u.TeV), magic_table_a4_lowzen['theta_68'],
             label='MAGIC, 2013, Zenith < 30°',  **mopt)
axes[2].plot(magic_table_a2['E_center'].to(u.TeV), magic_table_a2['bias'],
             ls='dotted',  **mopt)
axes[2].plot(magic_table_a2['E_center'].to(u.TeV), magic_table_a2['σ'],
             label='MAGIC, 2013, Zenith < 30°)', ls='-', **mopt)


for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))

axes[0].set_title(f'Efficiency {efficiency*100:.0f}%')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[0].set_ylim(*angresylim)
axes[1].set_ylim(*effareaylim)
axes[2].set_ylim(*eneylim)
axes[2].set_ylabel(r'$\Delta E/E_{true}$')
axes[2].legend()

axes[1].grid(True, which='both', axis='x')
axes[1].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')
    
axes[1].set_axisbelow(True)

axes[1].legend(loc='lower right')
plt.tight_layout()

outfile = Path(outdir, f'irfs_zeniths_eff_{efficiency:.2f}.png')
plt.savefig(outfile, dpi=200)


    
  

In [ ]:
import matplotlib.colors as colors
import matplotlib.lines as mlines

def plot_n_events(n_events, ax=None, **kwargs):
    ax = plt.gca() if ax is None else ax
    
    if 'color' not in kwargs:
        kwargs['color'] =  plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
    if 'ls' in kwargs:
        kwargs.pop('ls')
    
    n_events = n_events[1:-1]
    Emean = np.sqrt(n_events['true_energy_min'] * n_events['true_energy_max']) 
    ax.errorbar(Emean, n_events[f'simulated_events'], 
                xerr=(Emean-n_events['true_energy_min'], n_events['true_energy_max']-Emean),
                ls='-',
                elinewidth=1,
                capsize=2,
                ecolor=colors.to_rgba(kwargs['color'], 0.5),
                **kwargs
               )
    kwargs.pop('label')
    ax.errorbar(Emean, n_events[f'triggered_events'], 
                capsize=2,
                # xerr=(Emean-n_events['true_energy_min'], n_events['true_energy_max']-Emean),
                ls='--', 
                **kwargs,
               )

    ax.errorbar(Emean, n_events[f'selected_events'], 
                capsize=2,
                # xerr=(Emean-n_events['true_energy_min'], n_events['true_energy_max']-Emean),
                ls='dotted', 
                **kwargs
               )
    
    # Add legend
    lines, labels = ax.get_legend_handles_labels()
    line1 = mlines.Line2D([], [], color='black', label='Simulated events')
    line2 = mlines.Line2D([], [], color='black', ls='--', label='Triggered events')
    line3 = mlines.Line2D([], [], color='black', ls='dotted', label='Selected events')
    lines = [line1, line2, line3] + lines
    labels = [line1.get_label(), line2.get_label(), line3.get_label()] + labels

    ax.legend(lines, labels)

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$E_{true}$  / TeV')
    ax.set_ylabel('Number of events')
    ax.grid(True)
    return ax


In [ ]:
for ii, zen in enumerate(zeniths):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    n_events = QTable.read(filename, 'HIST_EVENTS')
    plot_n_events(n_events, color=calt[ii], label=f"zenith {file.stem.split('_')[2]}\u00B0")


outfile = Path(outdir, f'number_events_eff_{efficiency:.2f}.png')
plt.savefig(outfile, dpi=200)


## Source dependent IRFs

In [ ]:
indir = 'irfs_srcdep'
outdir = 'irfs_srcdep_plots'

Path(outdir).mkdir(exist_ok=True)

In [ ]:
zeniths

In [ ]:
zen = 10.00

fig, axes = plt.subplots(2, 1, figsize=(4, 4), sharex=True)


for ii, efficiency in enumerate(efficiencies):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}_srcdep.fits.gz')
    print_date(filename)

    plot_irfs.plot_effective_area_from_file(filename, ax=axes[0],
                                            ls='-',
                                            label=f'efficiency = {100*efficiency:.0f}%',
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='-',
                                               color=colors[ii],
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=colors[ii],
                                         ls='dotted',
                                         **opt
                                        )
    

plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                           ls='-',
                                           color=colors[ii],
                                           label=r'Energy resolution ($\sigma_{68}$)',
                                           **opt
                                              )
plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=colors[ii],
                                         ls='dotted',
                                     label=r'Energy bias (median)',
                                     **opt
                                    )


for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))

axes[0].set_title(f'Zenith {zen:.2f}°')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[0].set_ylim(*effareaylim)
axes[1].set_ylim(*eneylim)
axes[1].set_ylabel(r'$\Delta E/E_{true}$')
axes[1].legend()

axes[0].grid(True, which='both', axis='x')
axes[0].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')
axes[0].set_axisbelow(True)

axes[0].legend(loc='lower right')
plt.tight_layout()

outfile = Path(outdir, f'irfs_srcdep_eff_zen_{zen:.2f}.png')
plt.savefig(outfile, dpi=200)

In [ ]:
efficiency = 0.7

fig, axes = plt.subplots(2, 1, figsize=(4, 4), sharex=True)
fig.subplots_adjust(hspace=0) 

calt = [ colors[1], colors[3], colors[4], colors[5]]


for ii, zen in enumerate(zeniths):
    print(zen)
    color = calt[ii]
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}_srcdep.fits.gz')
    print_date(filename)

    plot_irfs.plot_effective_area_from_file(filename, ax=axes[0],
                                            ls='-',
                                            label=f'zenith = {zen:.2f}°',
                                            color=color,
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='-',
                                               color=color,
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=color,
                                         ls='dotted',
                                         **opt
                                        )
    

plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                           ls='-',
                                           color=color,
                                           label=r'Energy resolution ($\sigma_{68}$)',
                                           **opt
                                              )
plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=color,
                                         ls='dotted',
                                     label=r'Energy bias (median)',
                                     **opt
                                    )


for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))

axes[0].set_title(f'Efficiency {efficiency*100:.0f}%')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[0].set_ylim(*effareaylim)
axes[1].set_ylim(*eneylim)
axes[1].set_ylabel(r'$\Delta E/E_{true}$')
axes[1].legend()

axes[0].grid(True, which='major', axis='x')
axes[0].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')
axes[0].set_axisbelow(True)

axes[0].legend(loc='lower right')
plt.tight_layout()

outfile = Path(outdir, f'irfs_srcdep_zeniths_eff_{efficiency:.2f}.png')
plt.savefig(outfile, dpi=200)


    
  

# comparison source-dep vs source-indep

In [ ]:
zeniths

In [ ]:
zen = zeniths[0]

effs = [0.9]

fig, axes = plt.subplots(2, 1, figsize=(4, 4), sharex=True)


### src_indep


indir = 'irfs'

for ii, efficiency in enumerate(effs):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    print_date(filename)

    plot_irfs.plot_effective_area_from_file(filename, ax=axes[0],
                                            ls='-',
                                            label=f'efficiency = {100*efficiency:.0f}%',
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='-',
                                               color=colors[ii],
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=colors[ii],
                                         ls='dotted',
                                         **opt
                                        )
    

    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='-',
                                               color=colors[ii],
                                               # label=r'Energy resolution ($\sigma_{68}$)',
                                               **opt
                                                  )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                             color=colors[ii],
                                             ls='dotted',
                                         # label=r'Energy bias (median)',
                                         **opt
                                        )




### src_dep

indir = 'irfs_srcdep'

for ii, efficiency in enumerate(effs):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}_srcdep.fits.gz')
    print_date(filename)

    plot_irfs.plot_effective_area_from_file(filename, ax=axes[0],
                                            ls='--',
                                            label=f'efficiency = {100*efficiency:.0f}%  src-dep',
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='--',
                                               color=colors[ii],
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                         color=colors[ii],
                                         ls='-.',
                                         **opt
                                        )
    

    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[1], 
                                               ls='--',
                                               color=colors[ii+1],
                                               # label=r'Energy resolution ($\sigma_{68}$)',
                                               **opt
                                                  )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[1],
                                             color=colors[ii+1],
                                             ls='-.',
                                         # label=r'Energy bias (median)',
                                         **opt
                                        )


for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))

axes[0].set_title(f'Zenith {zen:.2f}°')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[0].set_ylim(*effareaylim)
axes[1].set_ylim(*eneylim)
axes[1].set_ylabel(r'$\Delta E/E_{true}$')
axes[1].legend()

axes[0].grid(True, which='both', axis='x')
axes[0].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')
    
axes[0].set_axisbelow(True)

axes[0].legend(loc='lower right')
plt.tight_layout()

outfile = Path(outdir, f'irfs_compare_srcdep_eff_zen_{zen:.2f}.png')
plt.savefig(outfile, dpi=200)